In [ ]:
import numpy as np
import pandas as pd 
import shutil, glob
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import imageio
import cv2

#Create a mergefile that is a combination of our individual state outfiles
with open('mergefile.txt', 'wb') as outfile:
    for filename in glob.glob('*.txt'):
        if filename == 'mergefile.txt':
            # don't want to copy the output into the output
            continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)

#Create our 
df = pd.DataFrame(pd.read_csv('mergefile.txt'))
df['isFlooded'] = 0
df[['Discharge', 'Date', 'Length of Record', 'Class', '% Normal Median', '% Normal Mean']] = df.flowinfo.str.split(";", expand = True)
df.lat = pd.to_numeric(df.lat, errors ='coerce').fillna(0).astype('float')
df.lng = pd.to_numeric(df.lng, errors ='coerce').fillna(0).astype('float')

In [ ]:
df

In [ ]:
#Add point to df using geopandas
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lng, df.lat), crs="EPSG:4326"
)

In [ ]:
import folium
import pandas as pd

#create a map
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.lat, point.lng],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

In [ ]:
# # # Instantiate Driver and import necessary libraries for web scraping
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.select import Select
# url= 'https://waterwatch.usgs.gov/index.php?id=flood&sid=w__table&r=us'
# driver = webdriver.Chrome()
# driver.get(url)

# #Get the table elements for the already flooded data
# flooded_rows = driver.find_elements(By.XPATH, "//tr")

# #List of the flood indicated USGS sites
# list_flooded = []
# for i in range(len(flooded_rows)):
#     list_flooded.append(flooded_rows[i].text)

# df2 = pd.DataFrame(list_flooded[5:])

# list_flooded = list_flooded[5:]

In [ ]:
# for i in range(len(list_flooded)):
#     if df['name'].str.contains(list_flooded[i][0:7]).any():
        

In [ ]:
list_flooded

In [ ]:
import json


m = folium.Map(tiles="Stamen Terrain", zoom_start=7, control_scale=True, max_bounds = True)
for _, row in df.iterrows():
    if row.Class == 'Class: High':
        folium.CircleMarker([row.lat, row.lng],
                          popup=row.name,
                          radius=1,
                          color = 'red',
                          ).add_to(m)
    elif row.Class == 'Class: 76-90':
        folium.CircleMarker([row.lat, row.lng],
                          popup=row.name,
                          radius=1,
                          color = 'orange',
                          ).add_to(m)
    elif row.Class == 'Class: 25-75':
        folium.CircleMarker([row.lat, row.lng],
                          popup=row.name,
                          radius=1,
                          color = 'yellow',
                          ).add_to(m)
    elif row.Class == 'Class: High':
        folium.CircleMarker([row.lat, row.lng],
                          popup=row.name,
                          radius=1,
                          color = 'purple',
                          ).add_to(m)
    else:
        folium.CircleMarker([row.lat, row.lng],
                          popup=row.name,
                          radius=1,
                          color = 'yellow',
                          ).add_to(m)
        
m

In [ ]:
import folium
import json

with open('gz_2010_us_050_00_500k.json') as handle:
    country_geo = json.loads(handle.read())

m = folium.Map(location = [df['lat'][5], df['lng'][5]],
               zoom_start=6,
               min_zoom = 5,
               max_zoom = 7)


folium.GeoJson(country_geo).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
from netCDF4 import Dataset
from mpl_toolkits import *